# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('clean_city_data.csv')
city_df.dropna(inplace = True)
city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,India,20.00,77.00,72.39,61,23,2.13,IN,1604077796
1,1,Ushuaia,-54.80,-68.30,50.00,71,75,28.86,AR,1604077571
2,2,Bhatkal,13.97,74.57,78.46,74,30,5.32,IN,1604077797
3,4,Longyearbyen,78.22,15.64,17.60,72,0,2.24,SJ,1604077720
4,5,Hermanus,-34.42,19.23,59.00,66,0,1.01,ZA,1604077798
...,...,...,...,...,...,...,...,...,...,...
558,2497,Matam,15.62,-13.33,102.20,20,0,4.70,SN,1604078379
559,2507,Santa Fe,-31.63,-60.70,69.01,40,100,8.01,AR,1604078183
560,2508,Matagami,49.75,-77.63,22.24,90,0,4.14,CA,1604078382
561,2513,Bousso,10.48,16.71,90.03,29,33,0.49,TD,1604078383


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]].astype(float)
# Store humidity 
weights = city_df["Humidity"].astype(float)
# Create a poverty Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 75) \
                                    & (city_df["Wind Speed"] < 10) \
                                    & (city_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,155,Sakakah,29.97,40.21,78.80,9,0,3.36,SA,1604077408
217,738,Halvad,23.02,71.18,79.65,34,0,7.07,IN,1604077972
276,1012,Makkah al Mukarramah,21.43,39.83,78.30,61,0,2.08,SA,1604078034
285,1057,Angoche,-16.23,39.91,79.84,74,0,9.98,MZ,1604078044
319,1207,Iranshahr,27.20,60.68,77.00,8,0,6.93,IR,1604078079
435,1806,Umluj,25.02,37.27,79.00,66,0,0.58,SA,1604078217
438,1830,Baghdad,33.34,44.40,77.00,27,0,4.70,IQ,1604078055
448,1875,Ceyhan,37.02,35.82,75.99,53,0,5.82,TR,1604078233
476,2044,Buraidah,26.33,43.98,77.00,22,0,5.82,SA,1604078272
543,2382,Najrān,17.49,44.13,75.20,23,0,9.17,SA,1604078351


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
69,Sakakah,SA,29.97,40.21,
217,Halvad,IN,23.02,71.18,
276,Makkah al Mukarramah,SA,21.43,39.83,
285,Angoche,MZ,-16.23,39.91,
319,Iranshahr,IR,27.20,60.68,
435,Umluj,SA,25.02,37.27,
438,Baghdad,IQ,33.34,44.40,
448,Ceyhan,TR,37.02,35.82,
476,Buraidah,SA,26.33,43.98,
543,Najrān,SA,17.49,44.13,


In [11]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
69,Sakakah,SA,29.97,40.21,Fakhamat Al Orjoana Apartment
217,Halvad,IN,23.02,71.18,Pop ciling
276,Makkah al Mukarramah,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
285,Angoche,MZ,-16.23,39.91,Guest House HESADA
319,Iranshahr,IR,27.20,60.68,کافی نت دانشگاه
435,Umluj,SA,25.02,37.27,Moon light Furnished Units
438,Baghdad,IQ,33.34,44.40,Baghdad Hotel
448,Ceyhan,TR,37.02,35.82,Ilbeyli Kiz Ogrenci Yurdu
476,Buraidah,SA,26.33,43.98,Mövenpick Hotel Qassim
543,Najrān,SA,17.49,44.13,محمد هادي ال فطيح


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))